<a href="https://colab.research.google.com/github/milox6m/MachinLerning2025/blob/main/chapter1/copy_of_untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import sys
import time
sys.path.insert(0, './..')

from openai import OpenAI

from dataset.edit_dataset import load_dataset
from utils.result import calculate_classification_metric, write_classification_metric
dataset = load_dataset('./../dataset/test.csv')
model = 'gpt-4-0125-preview'
prompt = [
    {
        'role': 'system',
        'content': '\
            You will be provided with a string. \
            Distinguish if the string causes SQL injection as a web security specialist.\
            If the string can cause SQL injection, return 1. If not, return 0.\
            Return only result for answer. Do not explain the result.'
    },
    {
        'role': 'user',
        'content': '<string>\' or \'a\' = \'a</string>'
    },
    {
        'role': 'assistant',
        'content': '1'
    },
    {
        'role': 'user',
        'content': '<string>Hello.</string>'
    },
    {
        'role': 'assistant',
        'content': '0'
    },
    {
        'role': 'user',
        'content': '<string>I am a Japanese<./string>'
    },
    {
        'role': 'assistant',
        'content': '0'
    },
    {
        'role': 'user',
        'content': '<string>-3756\' )  union all select 1034,1034,1034--</string>'
    },
    {
        'role': 'assistant',
        'content': '1'
    }
]
classification_counts = {
    'true_positives': 0,
    'false_positives': 0,
    'true_negatives': 0,
    'false_negatives': 0
}
inference_time = []
labels = []
raw_queries = []
preds = []

client = OpenAI()

for i, (query, label) in enumerate(dataset[:500]):
    print('i', i)
    prompt.append(
        {
            'role': 'user',
            'content': f'<string>{query}</string>'
        })

    start_time = time.time()
    chat_completion = client.chat.completions.create(
        model=model,
        messages=prompt,
        temperature=0.1,
        max_tokens=2
    )
    inference_time.append(time.time() - start_time)

    response = chat_completion.choices[0].message.content
    print('response:', response)
    try:
        result = int(re.search(r'(\d+)', response).group(1))
    except ValueError:
        result = 'N/A'
    except AttributeError:
        result = 'N/A'

    labels.append(int(label))
    raw_queries.append(query)
    preds.append(result)


    print(result, label)
    prompt.pop(9)

calculate_classification_metric(classification_counts, labels, preds, raw_queries)
write_classification_metric(classification_counts, inference_time)

ModuleNotFoundError: No module named 'dataset'